Trabalho apresentado ao curso [BI MASTER](https://ica.puc-rio.ai/bi-master) como pré-requisito para conclusão de curso e obtenção de crédito na disciplina "Projetos de Sistemas Inteligentes de Apoio à Decisão"

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## Leitura da NF

In [ ]:
if (True):
  import os
  os.chdir("drive/My Drive/PROJ/")  #MAPEAR PARA O LOCAL NO DRIVE ONDE SE ENCONTRA ESSE ARQUIVO

In [ ]:
!ls #Ver os arquivos

### Utilizando a biblioteca para "colar" por cima o texto decodificado - [OCRmyPDF](https://github.com/jbarlow83/OCRmyPDF)

In [ ]:
!sudo apt-get install ghostscript
!sudo apt install tesseract-ocr

In [ ]:
!pip install pluggy==0.13.0
!pip install folium==0.2.1
!pip install ocrmypdf

In [ ]:
!sudo apt-get update

In [ ]:
# Após a instalação do pluggy reiniciar o ambiente de execução para que o mesmo carregue a versão instalada!
import pluggy
import folium
import ocrmypdf
# import pytesseract  

In [ ]:
pluggy.__version__

'0.13.0'

In [ ]:
ocrmypdf.ocr('NF MATHEUS PSICO SET.pdf','Text_NF MATHEUS PSICO SET.pdf')

Scanning contents: 100%|██████████| 1/1 [00:00<00:00,  9.41page/s]
OCR: 100%|██████████| 1.0/1.0 [00:03<00:00,  3.42s/page]
PDF/A conversion: 100%|██████████| 1/1 [00:00<00:00,  5.06page/s]
JPEGs: 0image [00:00, ?image/s]
JBIG2: 0item [00:00, ?item/s]


<ExitCode.ok: 0>

### Utilizando a biblioteca [Apache Tika](https://tika.apache.org/0.5/documentation.html)

In [ ]:
!pip install tika

In [ ]:
from tika import parser

In [ ]:
try: 
  file_data = parser.from_file('Text_NF MATHEUS PSICO SET.pdf')
except:
  print("ERRO NA LEITURA DO ARQUIVO DE ENTRADA")

In [ ]:
text = file_data['content']
print(text)

In [ ]:
print(text.splitlines())
textlines = [textline for textline in text.splitlines() if textline != '' and textline != ' ']
textlines

### Percorre o contéudo do pdf para coletar as informações necessárias para solicitação do reembolso


In [ ]:
achouNumNF = False
achouPrestador = False
contLinhaPrestador = 1
achouDiscrim = False
nomeBeneficiario = "Rafael"

for textline in text.splitlines():
  if textline != '' and textline != ' ':

    if ("Numero da Nota" or "Número da Nota") in textline:
      achouNumNF = True
      continue
    if achouNumNF:
      notaFiscal = textline.split(" ")[0]
      try:
        print("notaFiscal=" + notaFiscal)
      except:
        print("ERRO: NOTA FISCAL")      
      emissaoNF = textline.split(" ")[1]
      try:
        print("emissaoNF=" + emissaoNF)
      except:
        print("ERRO: DATA EMISSÃO NF") 
      achouNumNF = False

    if "PRESTADOR" in textline:   
      achouPrestador = True
      continue
    if achouPrestador:
      if "CPF/CNPJ" in textline:
        cnpjCpf = textline.split(" ")[1]
        try:
          print("cnpjCpf=" + cnpjCpf)
        except:
          print("ERRO: CNPJ/CPF")         
      if ("Razao Social" or "Razão Social") in textline:
        razaoSocial = textline[19:]
        try:
          print("razaoSocial=" + razaoSocial) 
        except:
          print("ERRO: RAZÃO SOCIAL")       
      if ("Municipio" or "Município") in textline:
        cidAtend = textline.split(" ")[1]
        try:
          print("cidAtend=" + cidAtend) 
        except:
          print("ERRO: CIDADE ATENDIMENTO")                   
        ufAtend = textline.split(" ")[3]
        try:
          print("ufAtend=" + ufAtend) 
        except:
          print("ERRO: UF ATENDIMENTO")            
      contLinhaPrestador += 1
      if contLinhaPrestador > 4:
        achouPrestador = False

    if "DISCRIM" in textline:   
      achouDiscrim = True
      continue
    if achouDiscrim:
      if ("Matheus") in textline:
        nomeBeneficiario = "Matheus"
      if ("Rafaella") in textline:
        nomeBeneficiario = "Rafaella"   
      if ("Lidiane") in textline:
        nomeBeneficiario = "Lidiane"

  if "TOTAL DA NOTA" in textline:
    valorNF = textline.split(" ")[6]
    print("nomeBeneficiario=" + nomeBeneficiario)    
    try:
      print("valorNF=" + valorNF) 
    except:
      print("ERRO: VALOR NF")     
    achouDiscrim = False
      
      

notaFiscal=202000000000140
emissaoNF=19/10/2020
cnpjCpf=31.313.146/0001-10
razaoSocial=ESTIMULO - ESPACO DE DESENVOLVIMENTOPSI
cidAtend=Niterdi
ufAtend=RJ
nomeBeneficiario=Matheus
valorNF=430,00


## Submeter a NF

### Instala todas as bibliotecas necessárias

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
#!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
!wget https://chromedriver.storage.googleapis.com/2.42/chromedriver_linux64.zip  && unzip chromedriver_linux64

--2021-04-19 23:01:37--  https://chromedriver.storage.googleapis.com/2.42/chromedriver_linux64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 172.253.62.128, 2607:f8b0:4004:c07::80
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|172.253.62.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4039043 (3.9M) [application/zip]
Saving to: ‘chromedriver_linux64.zip.2’

chromedriver_linux6 100%[===================>]   3.85M  --.-KB/s    in 0.1s    

2021-04-19 23:01:37 (34.3 MB/s) - ‘chromedriver_linux64.zip.2’ saved [4039043/4039043]

Archive:  chromedriver_linux64.zip
replace chromedriver? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: chromedriver            


In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--start-maximized')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36')
chrome_options.add_argument('--ignore-certificate-errors')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options, service_args=['--verbose', '--log-path=/tmp/chromedriver.log'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


### Entrar com login e senha

In [ ]:
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
wd.delete_all_cookies()

In [ ]:
# Link para entrada de reembolso na AMS
wd.get("https://ams.petrobras.com.br/main.jsp?lumPageId=8A9E308F6524D1B50165A9CF9C6902DA")
time.sleep(2)

In [ ]:
matriculaNr = wd.find_element_by_xpath('//input[@name="PEAU_NR_MATRICULA_SUPERIOR"]')
matriculaNr.send_keys("010992552300")

senhaTx = wd.find_element_by_xpath('//input[@name="PEAU_TX_SENHA_SUPERIOR"]')
senhaTx.send_keys(open('password.txt').read()+Keys.ENTER)

time.sleep(2)

### Inputar os dados da NF


In [ ]:
bimaster_page = wd.page_source
bimaster_page_soup = BeautifulSoup(bimaster_page,'html.parser')
print(bimaster_page_soup)

In [ ]:
# Número da Carteirinha do Beneficiário [Nome]
numCarteirinha = "01.09925523.00" #Rafael
if nomeBeneficiario=="Matheus":
  numCarteirinha = "01.09925523.05"
if nomeBeneficiario=="Rafaella":
  numCarteirinha = "01.09925523.04"  
nomeNr = wd.find_element_by_xpath('//select[@name="PSLE_NM_NOME"]')
nomeNr.send_keys(numCarteirinha)

# Recibo (1) ou Nota Fiscal (2)
tipoDocTX = wd.find_element_by_xpath('//select[@name="PSLE_TX_TIPO_DOCUMENTO"]')
tipoDocTX.send_keys("2")

# Número da Nota Fiscal
#notaFiscal = "202000000000140"
documentoNr = wd.find_element_by_xpath('//input[@name="PSLE_NR_DOCUMENTO"]')
documentoNr.send_keys(notaFiscal)

# Data de Emissão da Nota Fiscal
#emissaoNF = "19/10/2020"
emissDocDt = wd.find_element_by_xpath('//input[@name="PSLE_DT_EMISS_DOCUMENTO"]')
emissDocDt.send_keys(emissaoNF)

# Tipo Pessoa (CPF ou CNPJ)
# tipoPessoa = "CNPJ"
if (len(cnpjCpf) > 14):
  tipoPessoa = "CNPJ"
else:
  tipoPessoa = "CPF"
tipoPessoaRd = wd.find_element_by_xpath('//input[@name="tipopessoa"]')
tipoPessoaRd.send_keys(tipoPessoa)

# CPF ou CNPJ
#cnpjCpf = "31.313.460/0011-0"
cnpjCpfNmRd = wd.find_element_by_xpath('//input[@name="PSLE_NM_CNPJCPF"]')
cnpjCpfNmRd.send_keys(cnpjCpf)

# Razão Social
#razaoSocial = "ESTIMULO - ESPAÇO DE DESENVOLVIMENTO PSI"
razSocNm = wd.find_element_by_xpath('//input[@name="PSLE_TX_RAZSOC_NOME"]')
razSocNm.send_keys(razaoSocial)

# UF do local do atendimento
#ufAtend = "RJ"
ufAtendTx = wd.find_element_by_xpath('//select[@name="PSLE_TX_UF_ATEND"]')
ufAtendTx.send_keys(ufAtend)

# Cidade do local do atendimento
#cidAtend = "NITERÓI"
if (cidAtend == "Niterdi"):
  cidAtend = "NITERÓI"
cidAtendTx = wd.find_element_by_xpath('//select[@name="PSLE_TX_CID_ATEND"]')
cidAtendTx.send_keys(cidAtend)

# Valor da NF
#valorNF = "430,00"
valorApNm = wd.find_element_by_xpath('//input[@name="PSLE_NM_VALOR_AP"]')
valorApNm.send_keys(valorNF)

# Data da Consulta
#dataConsulta = "19/10/2020"
dataConsulta = emissaoNF
consMedicaDt = wd.find_element_by_xpath('//input[@name="PSLE_DT_CONS_MEDICA"]')
consMedicaDt.send_keys(dataConsulta)

In [ ]:
# Arquivo
try: 
  fileNm = wd.find_element_by_xpath('//input[@name="PSLE_NM_FILE"]')
  fileNm.send_keys("C:\\Dropbox\\BI Master\\PROJ\\NF MATHEUS PSICO SET.pdf")
  fileNm.send_keys(Keys.ENTER)
  status = "OK"
except :
  print("ERRO NO ENVIO DOS DADOS")  




In [ ]:
wd.quit()

## Gravar Dados

In [ ]:
from sqlalchemy import create_engine
my_conn=create_engine("sqlite:////content/drive/MyDrive/my_db/my_db.db")
print("Conectado ao banco de dados com sucesso")

Conectado ao banco de dados com sucesso


### Criação da tabela

In [ ]:
from sqlalchemy.exc import SQLAlchemyError
try:
    my_conn.execute('''
        CREATE TABLE IF NOT EXISTS Reembolso(idReemb text primary key, 
                      numCarteirinha text, 
                      nomeBeneficiario text, 
                      notaFiscal text, 
                      emissaoNF text,
                      tipoPessoa text, 
                      cnpjCpf text, 
                      razaoSocial text,
                      ufAtend text, 
                      cidAtend text, 
                      valorNF real, 
                      dataConsulta text, 
                      status text 
                      );''')  
except SQLAlchemyError as e:
  #print(e)
  error = str(e.__dict__['orig'])
  print(error)
else:
  print("Tabela criada com sucesso...")

Tabela criada com sucesso...


### Inserção do registro

In [ ]:
try:
  r_set=my_conn.execute('''INSERT INTO `Reembolso` 
  (`idReemb`, `numCarteirinha`, `nomeBeneficiario`, `notaFiscal`, `emissaoNF`, `tipoPessoa`, `cnpjCpf`, `razaoSocial`, `ufAtend`, `cidAtend`, `valorNF`, `dataConsulta`, `status`) 
  VALUES  ("''' + notaFiscal +  '''", "'''  + numCarteirinha + '''", "''' + nomeBeneficiario + '''", "''' + notaFiscal + '''", "''' + emissaoNF + '''", "''' + tipoPessoa + '''", "''' + cnpjCpf + '''", "''' + razaoSocial + '''", "''' + ufAtend + '''", "''' + cidAtend + '''", "''' + valorNF + '''", "''' + dataConsulta + '''", "''' + status + '''");''')
except SQLAlchemyError as e:
  #print(e)
  error = str(e.__dict__['orig'])
  print(error)
else:
  print("Qtd. de registros adicionados : ",r_set.rowcount)

Qtd. de registros adicionados :  1


### Exibir os registros gravados

In [ ]:
# Exibir os registros gravados
r_set2=my_conn.execute('''SELECT * from Reembolso''');
for row in r_set2:
    print(row)

('202000000000140', '01.09925523.05', 'Matheus', '202000000000140', '19/10/2020', 'CNPJ', '31.313.146/0001-10', 'ESTIMULO - ESPACO DE DESENVOLVIMENTOPSI', 'RJ', 'NITERÓI', '430,00', '19/10/2020', 'OK')


### Para reiniciar uma nova tabela zerada

In [ ]:
# Para reiniciar uma nova tabela zerada
try:
  my_conn.execute(''' DROP table Reembolso ''')
except SQLAlchemyError as e:
  error=str(e.__dict__['orig'])
  print(error)
else:
  print('Tabela apagada')
finally:  
  my_conn.dispose()

Tabela apagada
